Train a simple deep CNN on the CIFAR10 small images dataset.

It gets down to 0.65 test logloss in 25 epochs, and down to 0.55 after 50 epochs, though it is still underfitting at that point.

In [ ]:
import keras
import numpy as np
keras.__version__

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [ ]:
# Parameters --------------------------------------------------------------
# It takes ~4 minutes to run each epoch, adjust the number to match time left
batch_size = 32
epochs = 2
data_augmentation = False

In [ ]:
# Data Preparation --------------------------------------------------------
from keras.datasets import cifar10
from keras.utils import to_categorical

# load the data
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

# Feature scale RGB values in test and train inputs  
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [ ]:
# Defining Model ----------------------------------------------------------

# Initialize sequential model
from keras import models
from keras import layers

model = models.Sequential()

# Start with hidden 2D convolutional layer being fed 32x32 pixel images
model.add(layers.Conv2D(32,(3,3), padding='same', input_shape=x_train.shape[1:]))
model.add(layers.Activation('relu'))

# Second hidden layer
model.add(layers.Conv2D(32,(3,3)))
model.add(layers.Activation('relu'))

# Use max pooling
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.25))

# 2 additional hidden 2D convolutional layers
model.add(layers.Conv2D(32,(3,3), padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(32,(3, 3)))
model.add(layers.Activation('relu'))

# Use max pooling once more
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.25))


# Flatten max filtered output into feature vector 
# and feed into dense layer
model.add(layers.Flatten())
model.add(layers.Dense(512))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))

# Outputs from dense layer are projected onto 10 unit output layer
model.add(layers.Dense(10))
model.add(layers.Activation('softmax'))

opt = keras.optimizers.RMSprop(lr=0.0001, decay = 1e-6)

# Compile the model
model.compile(optimizer=opt,
                loss='categorical_crossentropy',
                metrics=['acc'])

In [ ]:
# Training ----------------------------------------------------------------
from keras.preprocessing.image import ImageDataGenerator

if not(data_augmentation):
    history = model.fit(
    x_train, y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (x_test, y_test),
    shuffle = True)  
else:  
    datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
    
    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(x_train)

    # fits the model on batches with real-time data augmentation:
    history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=int(50000/batch_size), epochs=epochs,
                        validation_data=(x_test, y_test))

In [ ]:
import matplotlib.pyplot as plt
import pylab
history_dict = history.history

f, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12), dpi= 80)

ax1.plot(history_dict['loss'], 'o--', label='Training')
ax1.plot(history_dict['val_loss'], 'o--', label='Validation')
ax1.set_xlabel('Number of Epocs')
ax1.set_ylabel('Loss')
ax1.legend()
ax2.plot(history_dict['acc'], 'o--', label='Training')
ax2.plot(history_dict['val_acc'], 'o--', label='Validation')
ax2.set_xlabel('Number of Epocs')
ax2.set_ylabel('Accuracy')
ax2.legend()